### 用更 padas 的方法來處理


In [1]:
import tqdm
import tarfile
import pandas
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL
import gzip
from urllib.request import urlopen
%matplotlib inline
matplotlib.style.use('seaborn-bright')
# progress bar
tqdm.tqdm.pandas()

# 檔案名稱格式
filename_format="M06A_{year:04d}{month:02d}{day:02d}.tar.gz".format
xz_filename_format="xz/M06A_{year:04d}{month:02d}{day:02d}.tar.xz".format
csv_format = "M06A/{year:04d}{month:02d}{day:02d}/{hour:02d}/TDCS_M06A_{year:04d}{month:02d}{day:02d}_{hour:02d}0000.csv".format

# 設定欄位名稱
M06A_fields = ['VehicleType',
               'DetectionTime_O','GantryID_O',
               'DetectionTime_D','GantryID_D ',
               'TripLength', 'TripEnd', 'TripInformation']
import datetime
# 用來解析時間格式
def strptime(x):
    return datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10]), 
                             int(x[11:13]), int(x[14:16]), int(x[17:19]) )

def parse_tripinfo(tripinfo):
    split1 = tripinfo.split("; ")
    return [(strptime(t), t[20:]) for t in split1]

In [2]:
## 讀取檔案
node_data=pandas.read_json('node_data.json')
speedlimit = pandas.read_json('speedlimit.json')
speedlimit_lookup = node_data.速限.to_dict()

In [3]:
# 打開剛才下載的檔案試試
data_config ={"year":2016, "month":12, "day":18}
tar = tarfile.open(filename_format(**data_config), 'r')

# 如果沒有下載，可以試試看 xz 檔案
#data_dconfig ={"year":2016, "month":11, "day":18}
#tar = tarfile.open(xz_filename_format(**data_config), 'r')

In [4]:
import datetime
# 用來解析時間格式
def strptime(x):
    return datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10]), 
                             int(x[11:13]), int(x[14:16]), int(x[17:19]) )

In [5]:
from itertools import repeat, chain
valid_datas = []
for hr in range(24):
    print("process hr={}".format(hr))
    csv = tar.extractfile(csv_format(hour=hr, **data_config))
    data = pandas.read_csv(csv, names=M06A_fields)
    print("資料大小", data.shape)
    data = data[data.TripEnd == 'Y']
    drop_labels = [k for k in data.columns if k not in ['VehicleType', 'TripInformation']]
    data.drop(drop_labels, axis=1, inplace=True)
    # build trip_data
    split_tripinfo = data.TripInformation.str.split("; ")
    list_tripseg = list(chain(*split_tripinfo))
    tripinfo_len = split_tripinfo.apply(len)    
    chain_id = chain(*(repeat(*x) for x in zip(data.index, tripinfo_len) ))
    chain_vt = chain(*(repeat(*x) for x in zip(data.VehicleType, tripinfo_len) ))
    trip_data = pandas.DataFrame.from_records(zip(chain_id, chain_vt, list_tripseg), columns=['trip_id', 'VehicleType', 'trip_info'])
    # compute trip_info
    trip_data['time'] = trip_data.trip_info.apply(strptime)
    trip_data['speed_limit'] = trip_data.trip_info.str[20:].apply(speedlimit_lookup.get)
    trip_data['highway']= trip_data.trip_info.str[20:23]
    trip_data['km']= trip_data.trip_info.str[23:27]
    trip_data['dir']= trip_data.trip_info.str[27:]
    trip_data = trip_data.drop('trip_info', axis=1)
    trip_data = trip_data.drop(trip_data[trip_data.km.str.startswith('R')].index)
    trip_data.km = trip_data.km.astype(float)/10
    trip_data['time_delta'] = trip_data.time.diff()    
    trip_data['km_delta'] = np.abs(trip_data.km.diff())
    trip_data['speed_limit'] = pandas.concat([trip_data.speed_limit, trip_data.speed_limit.shift(1)], axis=1).max(axis=1)
    trip_data['same_highway'] = (trip_data.highway == trip_data.highway.shift()) & \
    (trip_data.dir == trip_data.dir.shift()) & (trip_data.trip_id == trip_data.trip_id.shift())
    valid_trip = trip_data.drop(trip_data[-trip_data.same_highway].index)
    del trip_data
    valid_trip['speed'] = valid_trip.km_delta * 3600 / valid_trip.time_delta.apply(lambda x: x.total_seconds())
    valid_trip['is_speeding'] = valid_trip.eval("speed > speed_limit+10")
    valid_trip.drop('same_highway', axis=1, inplace=1)
    valid_datas.append(valid_trip)


process hr=0
資料大小 (49270, 8)
process hr=1
資料大小 (30907, 8)
process hr=2
資料大小 (21924, 8)
process hr=3
資料大小 (19492, 8)
process hr=4
資料大小 (25457, 8)
process hr=5
資料大小 (42288, 8)
process hr=6
資料大小 (74356, 8)
process hr=7
資料大小 (124572, 8)
process hr=8
資料大小 (159241, 8)
process hr=9
資料大小 (188494, 8)
process hr=10
資料大小 (211247, 8)
process hr=11
資料大小 (208696, 8)
process hr=12
資料大小 (185384, 8)
process hr=13
資料大小 (206212, 8)
process hr=14
資料大小 (232777, 8)
process hr=15
資料大小 (240097, 8)
process hr=16
資料大小 (234978, 8)
process hr=17
資料大小 (223697, 8)
process hr=18
資料大小 (191004, 8)
process hr=19
資料大小 (183922, 8)
process hr=20
資料大小 (179076, 8)
process hr=21
資料大小 (153505, 8)
process hr=22
資料大小 (104948, 8)
process hr=23
資料大小 (59062, 8)


In [8]:
valid_trip = pandas.concat(valid_datas, ignore_index=True)
del valid_datas

In [9]:
valid_trip.speed.describe()

count    1.407309e+07
mean     9.326808e+01
std      2.160265e+01
min      1.215028e+00
25%      8.737864e+01
50%      9.800000e+01
75%      1.064452e+02
max      2.546341e+02
Name: speed, dtype: float64

In [10]:
# 有超速的路段平均
valid_trip.is_speeding.mean()

0.079701833762296254